In [1]:
import pandas as pd
df = pd.read_csv('input/listings_final.csv')

/tmp/ipykernel_82457/439669119.py:2: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('input/listings_final.csv')


In [2]:
df.columns

Index(['name', 'host_id', 'host_name', 'neighbourhood', 'latitude',
       'longitude', 'room_type', 'price', 'minimum_nights',
       'number_of_reviews', 'last_review', 'reviews_per_month',
       'calculated_host_listings_count', 'availability_365', 'property_type',
       'accommodates', 'first_review', 'review_scores_value',
       'review_scores_cleanliness', 'review_scores_location',
       'review_scores_accuracy', 'review_scores_communication',
       'review_scores_checkin', 'review_scores_rating', 'maximum_nights',
       'listing_url', 'host_is_superhost', 'host_about', 'host_response_time',
       'host_response_rate', 'street', 'weekly_price', 'monthly_price',
       'market'],
      dtype='object')

In [3]:
#Seleccionamos las variables que nos interesan en un nuevo df
df_pred = df.copy()
df_pred_new = df_pred[['neighbourhood', 'property_type', 'accommodates', 'room_type', 'price', 'maximum_nights', 'minimum_nights']]

df_pred_new.head() 

,neighbourhood,property_type,accommodates,room_type,price,maximum_nights,minimum_nights
0,Oostelijk Havengebied - Indische Buurt,Apartment,2,Private room,59,15,3
1,Westerpark,Apartment,5,Entire home/apt,160,20,4
2,Centrum-Oost,Townhouse,2,Entire home/apt,80,1000,1
3,Centrum-West,Apartment,3,Entire home/apt,125,60,14
4,Centrum-West,Houseboat,2,Private room,150,730,2


In [4]:
df_pred_new["minimum_nights"].unique()

array([   3,    4,    1,   14,    2,   28,    5,   30,    7,   90,    6,
         29,   13,   10,   21,  180,   60,   20,   15,   50,  183,    8,
         25,   31,   12,    9,   22,   40,  999,  100,  365,  186,  185,
        222,   26,   59,   18,   95,   19,   27,   70,   11,   45,  181,
        300, 1001,   52,   24,   17,  230,   42])

In [5]:
#importamos pycaret regression para calcular el price
from pycaret.regression import *

In [6]:
#hacemos un setup con visa  a nuestro target price
setup = setup(df_pred_new, target = 'price')

,Description,Value
0,Session id,7647
1,Target,price
2,Target type,Regression
3,Original data shape,"(20030, 7)"
4,Transformed data shape,"(20030, 30)"
5,Transformed train set shape,"(14021, 30)"
6,Transformed test set shape,"(6009, 30)"
7,Numeric features,3
8,Categorical features,3
9,Preprocess,True


In [7]:
#Evaluamos modelos
best = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
br,Bayesian Ridge,48.1484,19690.5709,120.9094,0.2682,0.3997,0.3326,0.0640
lr,Linear Regression,48.2902,19691.4652,120.9160,0.2681,0.4073,0.3342,0.2980
ridge,Ridge Regression,48.2840,19691.2937,120.9150,0.2681,0.4076,0.3341,0.0620
lasso,Lasso Regression,48.5815,19848.8634,121.6695,0.2569,0.3933,0.3395,0.0720
llar,Lasso Least Angle Regression,48.5815,19848.8647,121.6695,0.2569,0.3933,0.3395,0.0620
huber,Huber Regressor,46.1160,20189.5700,123.2867,0.2326,0.3783,0.2926,0.1820
en,Elastic Net,50.8348,20346.9712,124.0188,0.2216,0.4186,0.3756,0.0590
lightgbm,Light Gradient Boosting Machine,48.3246,20337.7068,124.4604,0.2100,0.3963,0.3310,0.1090
gbr,Gradient Boosting Regressor,48.1478,21111.0904,127.0149,0.1622,0.3881,0.3343,0.2570
knn,K Neighbors Regressor,53.8821,23047.1491,135.6958,0.0272,0.4300,0.3804,0.0960


In [8]:
#Vemos que el que mejor nos viene es lr 
lr = create_model('lr') # lo creamos

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,45.7293,6688.1441,81.7811,0.3312,0.4030,0.3237
1,47.4624,6798.8909,82.4554,0.3602,0.3853,0.3348
2,43.0369,4313.2539,65.6754,0.4524,0.3685,0.3181
3,51.8252,28671.1982,169.3257,0.1644,0.4091,0.3354
4,46.6770,10472.9313,102.3373,0.1973,0.4380,0.3506
5,57.1994,99508.3751,315.4495,0.0289,0.4260,0.3470
6,47.6945,17880.4444,133.7178,0.1570,0.3988,0.3294
7,48.1713,9996.0339,99.9802,0.2740,0.4211,0.3275
8,46.7192,5638.0504,75.0870,0.3665,0.4081,0.3324


In [9]:
# lo tuneamos XD
tuned_lr = tune_model(lr)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,45.7293,6688.1441,81.7811,0.3312,0.4030,0.3237
1,47.4624,6798.8909,82.4554,0.3602,0.3853,0.3348
2,43.0369,4313.2539,65.6754,0.4524,0.3685,0.3181
3,51.8252,28671.1982,169.3257,0.1644,0.4091,0.3354
4,46.6770,10472.9313,102.3373,0.1973,0.4380,0.3506
5,57.1994,99508.3751,315.4495,0.0289,0.4260,0.3470
6,47.6945,17880.4444,133.7178,0.1570,0.3988,0.3294
7,48.1713,9996.0339,99.9802,0.2740,0.4211,0.3275
8,46.7192,5638.0504,75.0870,0.3665,0.4081,0.3324


Fitting 10 folds for each of 2 candidates, totalling 20 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [10]:
#Veamos como se comporta
evaluate_model(tuned_lr)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [11]:
pred_df = predict_model(tuned_lr)
print (tuned_lr)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Linear Regression,48.3499,13331.4475,115.4619,0.1926,0.4300,0.3330


LinearRegression(n_jobs=-1)


In [12]:
#finalizamos el modelo
final_lr_best = finalize_model(tuned_lr)

In [13]:
#guardamos el modelo 
save_model( final_lr_best, 'ml_airbnb')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=FastMemory(location=/tmp/joblib),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['accommodates', 'maximum_nights',
                                              'minimum_nights'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=['neighbourhood', 'property_type',
                                              'room_type'],
                                     transformer=SimpleImputer(strategy='most_frequent'))),
                 ('onehot_encodi...
                  TransformerWrapper(include=['neighbourhood', 'room_type'],
                                     transformer=OneHotEncoder(cols=['neighbourhood',
                                                                     'room_type'],
                                                               handle_missing='return_nan',
                                                  